In [40]:
import pandas as pd
path = 'C://Users//User//Documents//GitHub//Springboard-DSC\\Take home challenges\\relax_challenge\\takehome_users.csv'
path_two = 'C://Users//User//Documents//GitHub//Springboard-DSC\\Take home challenges\\relax_challenge\\takehome_user_engagement.csv'
takehome_users = pd.read_csv(path)
takehome_engagement = pd.read_csv(path_two)

In [17]:
takehome_engagement = takehome_engagement.reset_index()
takehome_engagements = pd.melt(takehome_engagement,'time_stamp')

In [33]:
engagement_wide = takehome_engagement.groupby('user_id')['time_stamp'].apply(list).apply(pd.Series)

In [76]:
takehome_engagement['time_stamp'] = pd.to_datetime(takehome_engagement['time_stamp'])
#extract week number
takehome_engagement['week'] = takehome_engagement['time_stamp'].dt.week
grouped = takehome_engagement.groupby(['week','user_id'])['time_stamp'].count().reset_index(name='nums')
users = grouped[grouped['nums'] >= 3]
users['adopted_user'] = 'Yes'

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [77]:
merged = takehome_users.merge(users[['user_id','adopted_user']],how='left',
                         left_on='object_id', right_on='user_id')

In [106]:
merged.creation_source.unique()
merged.loc[(merged['creation_source'] == 'GUEST_INVITE') 'creation_source'] = 1
merged.loc[(merged['creation_source'] == 'ORG_INVITE') 'creation_source'] = 2
merged.loc[(merged['creation_source'] == 'SIGNUP') 'creation_source'] = 3
merged.loc[(merged['creation_source'] == 'PERSONAL_PROJECTS') 'creation_source'] = 4
merged.loc[(merged['creation_source'] == 'SIGNUP_GOOGLE_AUTH') 'creation_source'] = 5
#time delta between creation time and last login date
merged['time_delta'] = (pd.to_datetime(merged['creation_time']) - pd.to_datetime(merged['last_session_creation_time'])).dt.days

array(['GUEST_INVITE', 'ORG_INVITE', 'SIGNUP', 'PERSONAL_PROJECTS',
       'SIGNUP_GOOGLE_AUTH'], dtype=object)

In [101]:
merged['adopted_user'] = merged['adopted_user'].fillna('No')
#convere adopted user to numeric variable
merged.loc[(merged['adopted_user'] == 'Yes'),'adopted'] = 1
merged.loc[(merged['adopted_user'] == 'No'),'adopted'] = 0

In [122]:
merged.corr()

,object_id,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,user_id,adopted,time_delta
object_id,1.000000,-0.006096,-0.046270,-0.037062,-0.015767,0.100353,1.000000,-0.008067,0.015591
last_session_creation_time,-0.006096,1.000000,0.006148,-0.000075,0.066840,0.006375,-0.017179,0.732040,0.033120
opted_in_to_mailing_list,-0.046270,0.006148,1.000000,0.478601,-0.031263,0.022517,-0.050961,0.014749,0.016795
enabled_for_marketing_drip,-0.037062,-0.000075,0.478601,1.000000,-0.024446,0.016430,-0.042295,0.004916,-0.016038
org_id,-0.015767,0.066840,-0.031263,-0.024446,1.000000,-0.087800,-0.021849,0.089171,-0.046384
invited_by_user_id,0.100353,0.006375,0.022517,0.016430,-0.087800,1.000000,0.131333,0.002422,0.006724
user_id,1.000000,-0.017179,-0.050961,-0.042295,-0.021849,0.131333,1.000000,NaN,0.010578
adopted,-0.008067,0.732040,0.014749,0.004916,0.089171,0.002422,NaN,1.000000,-0.401994
time_delta,0.015591,0.033120,0.016795,-0.016038,-0.046384,0.006724,0.010578,-0.401994,1.000000


#### Analysis

From looking at the correlation matrix it would appear as though the numeric variable that is the biggest predicted of whether a user will convert to an adopted user is the time and date of their last login. This to me indicates that it might be interesting to create a new column that is delta between the time someone signed up and their last login. 